# Visualizando los datos de los reportes diarios
> Como podemos hacer para aprovechar los datos de los informes diarios

- toc: false 
- badges: true
- comments: false
- categories: [graficos]
- image: images/evolucion-por-provincia.png

En este post evaluamos rapidamente como se puede aprovechar la informacion que se encuentra disponible en el sitio https://www.argentina.gob.ar/coronavirus/informe-diario para tener una vision general de lo que pasa en el pais.

In [1]:
#hide

import arg_data
import pandas as pd

In [2]:
#hide

df = arg_data.get_arg_df()
cnt_by_place = df.groupby('infered_place').infected.sum()

Processing cache/11-04-20_reporte_vespertino_covid_19.pdf...
	Has 2 matches
Processing cache/10-04-20_reporte_vespertino_covid_19_0.pdf...
	Has 26 matches
Processing cache/09-04-20_reporte-vespertino-covid-19.pdf...
	Has 26 matches
Processing cache/08-04-20-reporte-vespertino-covid-19.pdf...


	Has 26 matches
Processing cache/07-04-20-reporte-vespertino-covid-19.pdf...
	Has 24 matches
Processing cache/06-04-20-reporte-vespertino-covid-19.pdf...
	Has 25 matches
Processing cache/05-04-20-reporte-vespertino-covid-19.pdf...


	Has 26 matches
Processing cache/04-04-20-reporte-vespertino-covid-19.pdf...
	Has 26 matches
Processing cache/03-04-20_reporte_vespertino_covid_19.pdf...
	Has 27 matches
Processing cache/02-04-20_reporte_vespertino_covid_19_0.pdf...


	Has 26 matches
Processing cache/01-04-20_reporte_vespertino_covid-19.pdf...
	Has 26 matches
Processing cache/31-03-20-reporte-vespertino-covid-19.pdf...
	Has 26 matches
Processing cache/30-03-20-reporte-vespertino-covid-19.pdf...
	Has 26 matches


Processing cache/29-03-20_reporte_vespertino_covid_19.pdf...
	Has 27 matches
Processing cache/28-03-20-reporte-vespertino-covid-19.pdf...
	Has 11 matches
Processing cache/27-03-20-reporte-diario-vespertino-covid-19.pdf...
	Has 15 matches
Processing cache/26-03-20-reporte-diario-vespertino-covid-19.pdf...


	Has 14 matches
Processing cache/25-03-20-reporte-diario-vespertino-covid-19.pdf...
	Has 15 matches
Processing cache/24-03-20-reporte-diario-vespertino-covid-19_0.pdf...
	Has 11 matches
Processing cache/23-03-2020-covid19_informe-diario.pdf...
	Has 8 matches
Processing cache/22-03-20-reporte-diario_covid19.pdf...
	Has 8 matches
Processing cache/21-03-20-reporte-diario_covid19.pdf...


	Has 12 matches
Processing cache/20-03-20_reporte_diario_covid_19_2.pdf...
	Has 11 matches
Processing cache/19-03-20-reporte-diario-covid-19.pdf...
	Has 9 matches
Processing cache/18-03-20_reporte_diario_covid-19_1.pdf...
	Has 8 matches
Processing cache/17-03-20_reporte_diario_covid-19.pdf...
	Has 12 matches
Processing cache/16-03-20-reporte-diario-covid-19_0.pdf...


	Has 7 matches
Skipping 15032020-nuevo-coronavirus-covid-19_0.pdf
Processing cache/14-03-20-reporte-diario-covid-19_0.pdf...
	Has 0 matches
Processing cache/13-03-20-reporte-diario-covid-19_0.pdf...
	Has 5 matches
Processing cache/12-03-20-nuevo-coronavirus-covid-19_reporte-diario_0.pdf...
	Has 0 matches
Processing cache/11-03-2020-nuevo-coronavirus-covid-19-reporte-diario_0.pdf...


	Has 3 matches
Processing cache/10-03-2020-nuevo-coronavirus-covid-19-reporte-diario_1.pdf...
	Has 1 matches
Processing cache/9-03-2020-nuevo-coronavirus-covid-19-reporte_diario_0.pdf...
	Has 1 matches
Processing cache/8-03-2020-nuevo-coronavirus-covid-19-reporte-diario_0.pdf...
	Has 1 matches
Processing cache/6-03-2020-nuevo-coronavirus-covid-19-reporte-diario_1.pdf...
	Has 1 matches
Processing cache/5-03-2020-nuevo-coronavirus-covid-19-reporte-diario_1.pdf...


	Has 0 matches


In [3]:
#hide
import altair as alt

def plot_evolution(selected_places=None, xaxis='date', xaxis_title='Fecha'):
    selection = alt.selection_multi(
        fields=['infered_place'], bind='legend', init=[{'infered_place': c} for c in (selected_places or [])]
    )
    
    return (
        alt.Chart(df)
           .mark_line(point=True, radius=150)
           .encode(
               x=alt.X(xaxis, axis=alt.Axis(title=xaxis_title)),
               y=alt.Y('cum_infected', scale=alt.Scale(type='log'), axis=alt.Axis(title='Casos confirmados')),
               color=alt.Color('infered_place', sort=list(cnt_by_place.index)),
               tooltip=list(df.columns), 
               opacity=alt.condition(selection, alt.value(.8), alt.value(.05))
            ).add_selection(selection)
             .configure_point(size=200)
             .properties(width=650, height=400)
             .configure_title(align='center', )
    )
    

# Evolucion en el tiempo de la cantidad de casos totales

Haciendo Shift+Click en las provincias que se ven a la derecha, podemos agregar o quitar provincias al gráfico

In [4]:
#hide_input
plot_evolution(cnt_by_place[cnt_by_place >=10].index.tolist())#, xaxis='days_from_first_infection')

alt.Chart(...)

# Graficando las mismas curvas cambiando el eje X

En lugar de utilizar la fecha, utilizamos cantidad de dias desde el primer caso confirmado

In [5]:
#hide_input
plot_evolution(
    cnt_by_place[cnt_by_place >=10].index.tolist(), 
    xaxis='days_from_first_infection', 
    xaxis_title='Dias desde el primer caso confirmado'
)

alt.Chart(...)

Los datos de este código ignoran los primeros informes, correspondiente a los primeros días de la infección. 

Los primeros informes, con los primeros casos eran mucho mas desestructurados, y es más dificultoso extraer la información. 

En los nuevos se itemiza por provincia, y como representan la gran mayoria de los casos, no están en estas estadísticas. 